In [1]:
pip install flask-cors


   ----- ---------------------------------- 1/8 [itsdangerous]
   ---------- ----------------------------- 2/8 [click]
   ---------- ----------------------------- 2/8 [click]
   ---------- ----------------------------- 2/8 [click]
   --------------- ------------------------ 3/8 [blinker]
   -------------------- ------------------- 4/8 [Werkzeug]
   -------------------- ------------------- 4/8 [Werkzeug]
   -------------------- ------------------- 4/8 [Werkzeug]
   -------------------- ------------------- 4/8 [Werkzeug]
   -------------------- ------------------- 4/8 [Werkzeug]
   -------------------- ------------------- 4/8 [Werkzeug]
   -------------------- ------------------- 4/8 [Werkzeug]
   -------------------- ------------------- 4/8 [Werkzeug]
   ------------------------- -------------- 5/8 [jinja2]
   ------------------------- -------------- 5/8 [jinja2]
   ------------------------- -------------- 5/8 [jinja2]
   ------------------------- -------------- 5/8 [jinja2]
   -------

In [2]:
pip install openai

  Using cached pydantic-2.11.7-py3-none-any.whl.metadata (67 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached typing_inspection-0.4.1-py3-none-any.whl.metadata (2.6 kB)
   ---------------------------------------- 0.0/812.0 kB ? eta -:--:--
   ---------------------------------------- 0.0/812.0 kB ? eta -:--:--
   ------------------------- -------------- 524.3/812.0 kB 1.5 MB/s eta 0:00:01
   ---------------------------------------- 812.0/812.0 kB 1.9 MB/s  0:00:00
Using cached pydantic-2.11.7-py3-none-any.whl (444 kB)
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/2.0 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/2.0 MB ? eta -:--:--
   ---------------- ----------------------- 0.8/2.0 MB 1.2 MB/s eta 0:00:01
   ---------------- ----------------------- 0.8/2.0 MB 1.2 MB/s eta 0:00:01
   ----------

In [ ]:
from flask import Flask, request, jsonify, render_template_string
from flask_cors import CORS
from openai import OpenAI
import json
import logging
from datetime import datetime
import os
from werkzeug.exceptions import BadRequest
import re
import uuid
import sys

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

app = Flask(__name__)
CORS(app, resources={r"/api/*": {"origins": "*"}})

# Configuration
class Config:
    OPENAI_API_KEY = "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"
    OPENAI_BASE_URL = "https://openrouter.ai/api/v1"
    MODEL_NAME = "microsoft/mai-ds-r1:free"
    MAX_TOKENS = 1000
    TEMPERATURE = 0.7
    SITE_URL = "http://localhost:5000"
    SITE_NAME = "Advanced AI Chatbot"

# Global client variable
client = None

def initialize_openai_client():
    """Initialize OpenAI client with proper error handling"""
    global client
    try:
        client = OpenAI(
            base_url=Config.OPENAI_BASE_URL,
            api_key=Config.OPENAI_API_KEY,
            timeout=30.0  # Add timeout
        )
        logger.info("OpenAI client initialized successfully")
        return True
    except Exception as e:
        logger.error(f"Failed to initialize OpenAI client: {str(e)}")
        client = None
        return False

# Initialize client on startup
client_initialized = initialize_openai_client()

# In-memory conversation storage
conversations = {}

class ChatbotService:
    @staticmethod
    def get_system_prompt():
        return """You are an advanced AI assistant chatbot. You are helpful, knowledgeable, and friendly.
        Provide clear, concise, and accurate responses. If you're unsure about something, acknowledge it.
        Always maintain a professional yet approachable tone."""
   
    @staticmethod
    def sanitize_input(text):
        """Basic input sanitization"""
        if not text or not isinstance(text, str):
            return ""
        # Remove excessive whitespace and limit length
        text = re.sub(r'\s+', ' ', text.strip())
        return text[:2000]
   
    @staticmethod
    def get_conversation(session_id):
        """Retrieve conversation history"""
        if session_id not in conversations:
            conversations[session_id] = {
                "messages": [{"role": "system", "content": ChatbotService.get_system_prompt()}],
                "created_at": datetime.now().isoformat(),
                "last_activity": datetime.now().isoformat()
            }
        return conversations[session_id]
   
    @staticmethod
    def add_message(session_id, role, content):
        """Add message to conversation"""
        conv = ChatbotService.get_conversation(session_id)
        conv["messages"].append({"role": role, "content": content})
        conv["last_activity"] = datetime.now().isoformat()
       
        # Keep only last 20 messages (excluding system message)
        if len(conv["messages"]) > 21:
            conv["messages"] = [conv["messages"][0]] + conv["messages"][-20:]
   
    @staticmethod
    def generate_response(session_id, user_message):
        """Generate AI response"""
        try:
            # Check if client is initialized
            if client is None:
                logger.error("OpenAI client is not initialized")
                return {"error": "AI service is not available. Please check configuration."}
           
            # Sanitize input
            user_message = ChatbotService.sanitize_input(user_message)
            if not user_message:
                return {"error": "Invalid or empty message"}
           
            # Add user message to conversation
            ChatbotService.add_message(session_id, "user", user_message)
            conv = ChatbotService.get_conversation(session_id)
           
            logger.info(f"Generating response for session: {session_id[:8]}...")
           
            # Make API call
            try:
                response = client.chat.completions.create(
                    extra_headers={
                        "HTTP-Referer": Config.SITE_URL,
                        "X-Title": Config.SITE_NAME,
                    },
                    model=Config.MODEL_NAME,
                    messages=conv["messages"],
                    max_tokens=Config.MAX_TOKENS,
                    temperature=Config.TEMPERATURE,
                )
               
                # Validate response
                if not response or not response.choices or len(response.choices) == 0:
                    logger.error("No response choices returned from API")
                    return {"error": "No response generated from AI model"}
               
                ai_response = response.choices[0].message.content
               
                if not ai_response:
                    logger.error("Empty response content from API")
                    return {"error": "Empty response from AI model"}
               
            except Exception as api_error:
                logger.error(f"OpenAI API error: {str(api_error)}")
                return {"error": f"AI service temporarily unavailable: {type(api_error).__name__}"}
           
            # Add AI response to conversation
            ChatbotService.add_message(session_id, "assistant", ai_response)
           
            logger.info(f"Response generated successfully for session: {session_id[:8]}...")
           
            return {
                "response": ai_response,
                "session_id": session_id,
                "timestamp": datetime.now().isoformat(),
                "model": Config.MODEL_NAME,
                "status": "success"
            }
           
        except Exception as e:
            logger.error(f"Unexpected error in generate_response: {str(e)}")
            return {
                "error": f"Service error: {type(e).__name__}",
                "timestamp": datetime.now().isoformat()
            }

# Enhanced HTML template
HTML_TEMPLATE = """
<!DOCTYPE html>
<html lang="en">
<head>
    <title>AI Chatbot - Test Interface</title>
    <meta charset="utf-8">
    <meta name="viewport" content="width=device-width, initial-scale=1">
    <style>
        * { box-sizing: border-box; }
        body {
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            margin: 0; padding: 20px;
            background-color: #f5f5f5;
        }
        .chat-container {
            max-width: 800px;
            margin: 0 auto;
            background: white;
            border-radius: 10px;
            box-shadow: 0 2px 10px rgba(0,0,0,0.1);
            overflow: hidden;
        }
        .header {
            background: #2196F3;
            color: white;
            padding: 20px;
            text-align: center;
        }
        .header h1 { margin: 0; }
        .status {
            padding: 10px 20px;
            background: #e8f5e8;
            border-left: 4px solid #4caf50;
            margin: 0;
        }
        .status.error {
            background: #ffeaea;
            border-left-color: #f44336;
            color: #d32f2f;
        }
        #messages {
            height: 400px;
            overflow-y: auto;
            padding: 20px;
            background: #fafafa;
        }
        .message {
            margin: 15px 0;
            padding: 12px 16px;
            border-radius: 18px;
            max-width: 80%;
            word-wrap: break-word;
        }
        .user {
            background: #2196F3;
            color: white;
            margin-left: auto;
            text-align: right;
        }
        .assistant {
            background: white;
            border: 1px solid #e0e0e0;
        }
        .error {
            background: #ffcdd2;
            color: #c62828;
            border: 1px solid #ef5350;
        }
        .input-area {
            padding: 20px;
            background: white;
            border-top: 1px solid #e0e0e0;
        }
        .input-group {
            display: flex;
            gap: 10px;
        }
        input[type="text"] {
            flex: 1;
            padding: 12px 16px;
            border: 1px solid #ddd;
            border-radius: 25px;
            outline: none;
            font-size: 14px;
        }
        input[type="text"]:focus {
            border-color: #2196F3;
        }
        button {
            padding: 12px 20px;
            border: none;
            border-radius: 25px;
            cursor: pointer;
            font-weight: 500;
            transition: all 0.2s;
        }
        .send-btn {
            background: #2196F3;
            color: white;
        }
        .send-btn:hover {
            background: #1976D2;
        }
        .send-btn:disabled {
            background: #ccc;
            cursor: not-allowed;
        }
        .clear-btn {
            background: #f44336;
            color: white;
        }
        .clear-btn:hover {
            background: #d32f2f;
        }
        .loading {
            display: none;
            color: #666;
            font-style: italic;
        }
    </style>
</head>
<body>
    <div class="chat-container">
        <div class="header">
            <h1>🤖 AI Chatbot</h1>
            <div>Test Interface</div>
        </div>
       
        <div id="status" class="status">
            System ready - Type a message to start chatting
        </div>
       
        <div id="messages"></div>
       
        <div class="input-area">
            <div class="input-group">
                <input
                    type="text"
                    id="messageInput"
                    placeholder="Type your message here..."
                    onkeypress="if(event.key==='Enter' && !event.shiftKey) { event.preventDefault(); sendMessage(); }"
                    maxlength="2000"
                >
                <button id="sendBtn" class="send-btn" onclick="sendMessage()">Send</button>
                <button class="clear-btn" onclick="clearChat()">Clear</button>
            </div>
            <div id="loading" class="loading">AI is thinking...</div>
        </div>
    </div>

    <script>
        const sessionId = 'session_' + Date.now() + '_' + Math.random().toString(36).substr(2, 9);
        let isProcessing = false;
       
        function updateStatus(message, isError = false) {
            const status = document.getElementById('status');
            status.textContent = message;
            status.className = isError ? 'status error' : 'status';
        }
       
        function addMessage(role, content, isError = false) {
            const messages = document.getElementById('messages');
            const messageDiv = document.createElement('div');
            messageDiv.className = 'message ' + (isError ? 'error' : role);
            messageDiv.textContent = content;
            messages.appendChild(messageDiv);
            messages.scrollTop = messages.scrollHeight;
        }
       
        function setLoading(loading) {
            const sendBtn = document.getElementById('sendBtn');
            const loadingDiv = document.getElementById('loading');
            const input = document.getElementById('messageInput');
           
            isProcessing = loading;
            sendBtn.disabled = loading;
            loadingDiv.style.display = loading ? 'block' : 'none';
           
            if (loading) {
                sendBtn.textContent = 'Wait...';
                input.disabled = true;
            } else {
                sendBtn.textContent = 'Send';
                input.disabled = false;
                input.focus();
            }
        }
       
        async function sendMessage() {
            const input = document.getElementById('messageInput');
            const message = input.value.trim();
           
            if (!message || isProcessing) return;
           
            addMessage('user', message);
            input.value = '';
            setLoading(true);
            updateStatus('Sending message...');
           
            try {
                const response = await fetch('/api/chat', {
                    method: 'POST',
                    headers: {
                        'Content-Type': 'application/json',
                    },
                    body: JSON.stringify({
                        message: message,
                        session_id: sessionId
                    })
                });
               
                const data = await response.json();
               
                if (!response.ok) {
                    throw new Error(`Server error: ${response.status}`);
                }
               
                if (data.error) {
                    addMessage('assistant', 'Error: ' + data.error, true);
                    updateStatus('Error occurred', true);
                } else {
                    addMessage('assistant', data.response);
                    updateStatus('Message sent successfully');
                }
               
            } catch (error) {
                console.error('Request failed:', error);
                addMessage('assistant', `Connection error: ${error.message}`, true);
                updateStatus('Connection failed', true);
            } finally {
                setLoading(false);
            }
        }
       
        async function clearChat() {
            if (!confirm('Are you sure you want to clear the chat?')) return;
           
            try {
                await fetch(`/api/conversation/${sessionId}`, {
                    method: 'DELETE'
                });
                document.getElementById('messages').innerHTML = '';
                updateStatus('Chat cleared');
            } catch (error) {
                console.error('Error clearing chat:', error);
                updateStatus('Failed to clear chat', true);
            }
        }
       
        // Initialize
        document.getElementById('messageInput').focus();
       
        // Check API health on load
        fetch('/api/health')
            .then(r => r.json())
            .then(data => {
                updateStatus(`System ready - AI service: ${data.ai_service || 'unknown'}`);
            })
            .catch(() => {
                updateStatus('Warning: Could not connect to API', true);
            });
    </script>
</body>
</html>
"""

# Routes
@app.route('/')
def index():
    """Serve the main chat interface"""
    return render_template_string(HTML_TEMPLATE)

@app.route('/api/chat', methods=['POST'])
def chat():
    """Main chat endpoint"""
    try:
        if not request.is_json:
            logger.warning("Received non-JSON request")
            return jsonify({"error": "Request must be JSON"}), 400
           
        data = request.get_json()
       
        if not data:
            logger.warning("Received empty request data")
            return jsonify({"error": "No data provided"}), 400
           
        if 'message' not in data:
            logger.warning("Received request without message field")
            return jsonify({"error": "Message field is required"}), 400
       
        user_message = data.get('message', '').strip()
        session_id = data.get('session_id', str(uuid.uuid4()))
       
        if not user_message:
            return jsonify({"error": "Message cannot be empty"}), 400
       
        logger.info(f"Processing chat request - Session: {session_id[:8]}..., Message: {user_message[:50]}...")
       
        # Generate response
        result = ChatbotService.generate_response(session_id, user_message)
       
        if 'error' in result:
            logger.error(f"Response generation failed: {result['error']}")
            return jsonify(result), 500
       
        logger.info(f"Chat request completed successfully for session: {session_id[:8]}...")
        return jsonify(result)
       
    except Exception as e:
        logger.error(f"Unexpected error in chat endpoint: {str(e)}", exc_info=True)
        return jsonify({
            "error": f"Internal server error: {type(e).__name__}",
            "timestamp": datetime.now().isoformat()
        }), 500

@app.route('/api/conversation/<session_id>', methods=['GET'])
def get_conversation_history(session_id):
    """Get conversation history"""
    try:
        conv = ChatbotService.get_conversation(session_id)
        messages = [msg for msg in conv["messages"] if msg["role"] != "system"]
        return jsonify({
            "messages": messages,
            "session_id": session_id,
            "created_at": conv["created_at"],
            "last_activity": conv["last_activity"],
            "message_count": len(messages)
        })
    except Exception as e:
        logger.error(f"Error retrieving conversation: {str(e)}")
        return jsonify({"error": f"Error retrieving conversation: {str(e)}"}), 500

@app.route('/api/conversation/<session_id>', methods=['DELETE'])
def clear_conversation(session_id):
    """Clear conversation history"""
    try:
        if session_id in conversations:
            del conversations[session_id]
            logger.info(f"Conversation cleared for session: {session_id[:8]}...")
        return jsonify({"message": "Conversation cleared successfully"})
    except Exception as e:
        logger.error(f"Error clearing conversation: {str(e)}")
        return jsonify({"error": f"Error clearing conversation: {str(e)}"}), 500

@app.route('/api/health', methods=['GET'])
def health_check():
    """Health check endpoint"""
    return jsonify({
        "status": "healthy",
        "timestamp": datetime.now().isoformat(),
        "version": "1.1.0",
        "ai_service": "available" if client else "unavailable",
        "active_conversations": len(conversations)
    })

@app.route('/api/test', methods=['GET', 'POST'])
def test_endpoint():
    """Test endpoint for debugging"""
    return jsonify({
        "message": "API is working correctly",
        "method": request.method,
        "timestamp": datetime.now().isoformat(),
        "config": {
            "model": Config.MODEL_NAME,
            "base_url": Config.OPENAI_BASE_URL,
            "api_key_configured": bool(Config.OPENAI_API_KEY and len(Config.OPENAI_API_KEY) > 10),
            "client_initialized": client is not None
        },
        "headers": dict(request.headers)
    })

@app.errorhandler(404)
def not_found(error):
    logger.warning(f"404 - Path not found: {request.path}")
    return jsonify({"error": "Endpoint not found", "path": request.path}), 404

@app.errorhandler(500)
def internal_error(error):
    logger.error(f"500 - Internal server error: {str(error)}")
    return jsonify({"error": "Internal server error"}), 500

def main():
    """Main function to run the app"""
    try:
        logger.info("=" * 50)
        logger.info("Starting Advanced AI Chatbot Server")
        logger.info("=" * 50)
       
        # Log configuration
        logger.info(f"Model: {Config.MODEL_NAME}")
        logger.info(f"Base URL: {Config.OPENAI_BASE_URL}")
        logger.info(f"API Key configured: {bool(Config.OPENAI_API_KEY)}")
        logger.info(f"Client initialized: {client_initialized}")
       
        if not client_initialized:
            logger.warning("WARNING: OpenAI client failed to initialize!")
            logger.warning("The server will start but AI features may not work.")
       
        # Start the server
        logger.info("Starting Flask server on http://0.0.0.0:5000")
        logger.info("Visit http://localhost:5000 to test the chatbot")
        logger.info("API endpoints available at /api/*")
       
        app.run(
            debug=False,  # Disable debug mode to prevent reloader issues
            host='0.0.0.0',
            port=5000,
            threaded=True
        )
       
    except KeyboardInterrupt:
        logger.info("\nServer stopped by user (Ctrl+C)")
    except Exception as e:
        logger.error(f"Failed to start server: {str(e)}", exc_info=True)
        sys.exit(1)

if __name__ == '__main__':
    main()

2025-08-28 12:37:18,283 - __main__ - INFO - OpenAI client initialized successfully
2025-08-28 12:37:18,294 - __main__ - INFO - ==================================================
2025-08-28 12:37:18,295 - __main__ - INFO - Starting Advanced AI Chatbot Server
2025-08-28 12:37:18,296 - __main__ - INFO - ==================================================
2025-08-28 12:37:18,297 - __main__ - INFO - Model: microsoft/mai-ds-r1:free
2025-08-28 12:37:18,298 - __main__ - INFO - Base URL: https://openrouter.ai/api/v1
2025-08-28 12:37:18,299 - __main__ - INFO - API Key configured: True
2025-08-28 12:37:18,300 - __main__ - INFO - Client initialized: True
2025-08-28 12:37:18,301 - __main__ - INFO - Starting Flask server on http://0.0.0.0:5000
2025-08-28 12:37:18,301 - __main__ - INFO - Visit http://localhost:5000 to test the chatbot
2025-08-28 12:37:18,302 - __main__ - INFO - API endpoints available at /api/*


 * Serving Flask app '__main__'
 * Debug mode: off


2025-08-28 12:37:18,313 - werkzeug - INFO - WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.1.50:5000
2025-08-28 12:37:18,315 - werkzeug - INFO - Press CTRL+C to quit
2025-08-28 12:37:37,012 - werkzeug - INFO - 127.0.0.1 - - [28/Aug/2025 12:37:37] "GET / HTTP/1.1" 200 -
2025-08-28 12:37:37,338 - werkzeug - INFO - 127.0.0.1 - - [28/Aug/2025 12:37:37] "GET /api/health HTTP/1.1" 200 -
2025-08-28 12:37:57,819 - __main__ - INFO - Processing chat request - Session: session_..., Message: hiiiii...
2025-08-28 12:37:57,824 - __main__ - INFO - Generating response for session: session_...
2025-08-28 12:38:03,511 - httpx - INFO - HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-28 12:38:05,591 - __main__ - INFO - Response generated successfully for session: session_...
2025-08-28 12:38